In [1]:
import time

import torch
import torch.nn as nn
from torch import nn, optim, Tensor
from torch.utils.data import DataLoader
import torch.nn.functional as F

import numpy as np
from typing import List, Tuple, Any, Optional, Dict

from data.raw_data import Instance, Data
from data.data_utils import Dictionary, triples_to_indices
from data.dataset import CLUTRRdata
from models.encoders import EncoderLSTM
from models.decoders import DecoderLSTM
from training.train import train, predict_raw

In [2]:
test_data_path = ['data/clutrr-data/data_089907f8/1.2_test.csv']
data = Data(test_paths = test_data_path)
dictionary = Dictionary(data)
story, target = data.train[5080].story, data.train[5000].target
s, p, o = target
print(f'Number of entities {dictionary.num_entities}')
print(f'Example story: {story}')
print(f'Example target: {target}')

entity_embeddings = nn.Embedding(dictionary.num_entities, 100, sparse=True)
relation_embeddings = nn.Embedding(dictionary.num_relations, 100, sparse=True)

# Currently no backprop through embeddings
entity_embeddings.weight.requires_grad = False
relation_embeddings.weight.requires_grad = False
print(f'entity embeddings shape: {entity_embeddings.weight.shape}')

indices, ent, rel = triples_to_indices(dictionary, story)

Number of entities 78
Example story: [('Dwight', 'brother', 'Christopher'), ('Christopher', 'daughter', 'Lucille')]
Example target: ('James', 'father', 'Jason')
entity embeddings shape: torch.Size([78, 100])


In [3]:
train_set = CLUTRRdata(data.train, dictionary)
test_set = CLUTRRdata(data.test, dictionary)

In [4]:
encoder = EncoderLSTM(hidden_size = 100,
                    entity_embeddings = entity_embeddings,
                    relation_embeddings = relation_embeddings)

decoder = DecoderLSTM(hidden_size = 100,
                    entity_embeddings = entity_embeddings,
                    num_relations = dictionary.num_relations)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = encoder.to(device)
decoder = decoder.to(device)

In [5]:
train(encoder,
      decoder,
      train_set,
      test_set,
      epochs = 4,
      learning_rate = 0.01)

Training model...
Initial values: 
Train loss: 155.06 | Train acc: 2.55% | Test loss: 163.07 | Test acc: 0.00%
Epoch [1/4],Train loss: 14.0012,Test loss: 27.3578 Train acc: 90.37% Test acc: 71.05%
Epoch [2/4],Train loss: 0.8007,Test loss: 43.0047 Train acc: 99.53% Test acc: 71.05%
Epoch [3/4],Train loss: 0.3904,Test loss: 35.3882 Train acc: 99.74% Test acc: 68.42%
Epoch [4/4],Train loss: 4.7297,Test loss: 6.9632 Train acc: 97.04% Test acc: 100.00%
Finished training



In [7]:
for i in range(10):
    print(predict_raw(data.test[i], encoder, decoder, dictionary))

Story: [('Jason', 'grandson', 'Scott'), ('Scott', 'brother', 'Lewis')]
Target: ('Jason', 'grandson', 'Lewis')
Predicted relation: grandson
None
Story: [('Gabrielle', 'husband', 'Jason'), ('Jason', 'daughter', 'Lisa')]
Target: ('Gabrielle', 'daughter', 'Lisa')
Predicted relation: daughter
None
Story: [('Gabrielle', 'husband', 'Jason'), ('Jason', 'daughter', 'Myrna')]
Target: ('Gabrielle', 'daughter', 'Myrna')
Predicted relation: daughter
None
Story: [('Gabrielle', 'grandson', 'David'), ('David', 'brother', 'Joe')]
Target: ('Gabrielle', 'grandson', 'Joe')
Predicted relation: grandson
None
Story: [('Jason', 'grandson', 'Donald'), ('Donald', 'brother', 'Russell')]
Target: ('Jason', 'grandson', 'Russell')
Predicted relation: grandson
None
Story: [('Dorothy', 'husband', 'James'), ('James', 'daughter', 'Theresa')]
Target: ('Dorothy', 'daughter', 'Theresa')
Predicted relation: daughter
None
Story: [('Myrna', 'husband', 'Christopher'), ('Christopher', 'daughter', 'Lucille')]
Target: ('Myrna', '